# "The Battle of the Neighborhoods" - Week 1
### Applied Data Science Capstone Project
<hr>

## Table of contents
* [Introduction/Business Problem](#introduction)
* [Data](#data)


<hr>

## Introduction/Business Problem <a name="introduction"></a>

##### Purpose
This project aims to identify and prioritize improvement opportunities for neighborhood parks in Green Bay, Wisconsin, USA.
##### Audience
The target audience for this analysis is a community action group working toward improved elementary education outcomes for the city population. The group includes representation from the school district, local government, business community leaders, and citizen stakeholders. The group wants to know if they should pursue a park improvement initiative.
##### Background
Successful school outcomes are the result of a complex system of factors. Many factors are within a school's control, such as student-to-teacher ratio or highly qualified teachers. Many more factors, however, are outside a school's control, such as parent engagement, neighborhood safety, and food security. To effectively address this complex issue, a complex solution is required, with interventions applied to internal and external factors concurrently. It is unrealistic to expect a school district to create meaningful and lasting improvements to student outcomes without outside assistance addressing external factors.
The external factor examined by this analysis is equitable access to outdoor recreational activities provided by public parks. The greenspaces and playgrounds found in public parks promote the holistic development of healthy children through physical play, environment exploration, and unstructured, self-directed play.

### Step 1: Proof of Principle
Based on observations made from the data, is this idea worth pursuing? The primary purpose of this analysis is to **determine whether or not to pursue a park improvement initiative.** In other words, is there any measurable relationship between school outcomes and access to parks.
### Step 2: Key Questions
If the determination is made to pursue the initiative, which park improvements should be prioritized?
* **Which neighborhoods** are in need of park improvement/development?
* **What specific improvements** might provide the most benefit to park users?
<hr>


## Data <a name="data"></a>

To understand the current state of elementary schools in Green Bay, the following data will be pulled from the Wisconsin Department of Public Instruction ("DPI") website:
* **Names and locations of elementary schools** in the Green Bay Area Public School District ("GBAPS").
* **School accountability reports** from the last seven years. These reports include a variety of performance metrics. The data will be used to select neighborhoods in greatest need of intervention. This data can also be used as a target for machine learning models seeking to explain factors related to school performance.

To understand the current state of parks in Green Bay, the following data will be retrieved from the Foursquare API.
* **Names and locations of parks** in the proximity of elementary schools. Park distance from school will be a feature considered for modeling.
* **User ratings of parks.** This will serve as a metric of park quality, another feature to be considered for modeling.


## School Data
#### Names and locations
First I'll grab the names and locations of schools. This data is easily acquired by downloading a CSV from the DPI public data portal. Let's load the CSV into a dataframe and clean it up.

In [1]:
import pandas as pd
import numpy as np

In [2]:
schools = pd.read_csv('school_list.csv')
schools.columns

Index(['X', 'Y', 'FID', 'SCHOOL', 'DISTRICT', 'SCH_TYPE', 'LOW_GRADE',
       'HIGH_GRADE', 'TYPE_DESC', 'SCH_CAT', 'CHARTER', 'VIRTUAL', 'FULL_ADDR',
       'STAND_ADDR', 'PHYS_COUNT', 'CESA', 'DIST_CODE', 'SCH_CODE', 'UNIQUE_',
       'NCES_DIST', 'NCES_SCH', 'ASM', 'SEN', 'CON', 'Latitude', 'Longitude'],
      dtype='object')

Let's filter out the data we won't use. First, unused columns will be dropped so the dataframe is easier to look at. We will keep the columns related to school name, type, grade level, address, and geocoordinates. Second, we'll focus on dropping rows that do not contain elementary schools.

In [3]:
# Drop the columns that will not be used
schools.drop(['X','Y','FID','DISTRICT','TYPE_DESC','SCH_CAT','CHARTER','VIRTUAL','FULL_ADDR','PHYS_COUNT','CESA','DIST_CODE','SCH_CODE','UNIQUE_','NCES_DIST','NCES_SCH','ASM','SEN','CON'], axis=1, inplace=True)

# Since we're focusing on elementary schools, keep rows with a starting grade of 'K' and an ending grade of 5 or 8.
# Note, K-8 schools are technically not considered elementary schools. I'll probably end up dropping them, but want to see the data first.
schools = schools[schools.LOW_GRADE.str.contains('K')]
schools = schools[(schools.HIGH_GRADE.str.contains('5') | schools.HIGH_GRADE.str.contains('8'))]

# Sort dataframe by school name and reset index
schools = schools.sort_values('SCHOOL')
schools.reset_index(inplace=True, drop=True)

In [4]:
print(f'The dataframe has {schools.shape[0]} rows and {schools.shape[1]} columns.')
schools.head()

The dataframe has 28 rows and 7 columns.


,SCHOOL,SCH_TYPE,LOW_GRADE,HIGH_GRADE,STAND_ADDR,Latitude,Longitude
0,Aldo Leopold Comm Sch K-8,Combined Elementary/Secondary School,K4,08,"622 Eliza St Green Bay, WI 54301-3229",44.499981,-88.017998
1,Baird El,Elementary School,K4,05,"539 Laverne Dr Green Bay, WI 54311-5720",44.510968,-87.933378
2,Beaumont El,Elementary School,K4,05,"1505 Gatewood St Green Bay, WI 54304-3375",44.505500,-88.051171
3,Chappell El,Elementary School,K4,05,"205 N Fisk St Green Bay, WI 54303-3199",44.530298,-88.054852
4,Danz El,Elementary School,K4,05,"2130 Basten St Green Bay, WI 54302-5398",44.508833,-87.968792


#### School accountability scores
The next piece of school data we need are performance metrics, which the DPI reports annually in School Report Cards. After much searching through the DPI data portal, I could only find report cards shared in PDF format. I bulk-downloaded all the PDFs and scraped them to populate a dataframe using two libraries: os and pdfminer. I may share this process in a separate notebook, but for the purpose of this analysis I'm only going to import the output.

*Note, the DPI did not publish school report cards for the 2014-2015 school year*

In [5]:
school_scores = pd.read_csv('school_report_cards.csv')
print(f'The dataframe has {school_scores.shape[0]} rows and {school_scores.shape[1]} columns.')
school_scores.head()

The dataframe has 193 rows and 7 columns.


,School,Year,Overall Score,Student Achievement,Student Growth,Closing Gaps,On-Track and Postsecondary Readiness
0,Aldo Leopold,2011-12,74.5,71.5,74.1,63.5,89.0
1,Aldo Leopold,2012-13,75.3,72.6,69.6,68.3,90.6
2,Aldo Leopold,2013-14,77.3,72.1,71.7,75.6,89.9
3,Aldo Leopold,2015-16,66.0,70.6,37.5,66.5,89.0
4,Aldo Leopold,2016-17,70.3,66.3,61.3,64.8,88.0


In [6]:
school_scores['Overall Score'][school_scores['School']=='Baird'].mean()

68.05714285714285

In [7]:
avg_scores=[]
for school in list(school_scores.School.unique()):
    avg_scores.append(school_scores['Overall Score'][school_scores['School']==school].mean())

In [8]:
import folium, geocoder
import matplotlib.cm as cm
import matplotlib.colors as colors

In [9]:
def coordinates(address):
    coords = None
    while (coords is None):
        g = geocoder.arcgis(address)
        coords = g.latlng   
    return coords

In [10]:
avg_scores=[]
for school in list(school_scores.School.unique()):
    avg_scores.append(school_scores['Overall Score'][school_scores['School']==school].mean())
plot_df = schools[['SCHOOL','Latitude','Longitude']].copy()
plot_df['AvgScore'] = avg_scores
plot_df['MarkerColor'] = pd.cut(plot_df['AvgScore'], bins=5, labels=['red','orange','yellow','green','blue'])

map = folium.Map(location=coordinates('Green Bay, WI'), zoom_start=12, tiles='Stamen Toner')
for i,row in plot_df.iterrows():
    folium.CircleMarker((row['Latitude'],row['Longitude']),
                         radius=10,
                         color=row['MarkerColor'],
                         fill=True,
                        popup=str(row['SCHOOL']+': '+str(round(row['AvgScore'],1)))).add_to(map)
map

In [11]:
plot_df[plot_df.AvgScore==plot_df.AvgScore.max()]

,SCHOOL,Latitude,Longitude,AvgScore,MarkerColor
16,Leonardo da Vinci Sch-Gifted,44.511129,-88.010443,97.825,blue


Clearly there is an outlier in the data. The school with a blue marker has an average score significantly higher than the others. That school is for gifted and talented students, so we'll remove that from the analysis. Let's go ahead and also remove the other K-8 schools too, so that we're dealing strictly with elementary schools.

In [12]:
plot_df.drop(plot_df.index[plot_df.SCHOOL.str.contains('El')==False], axis=0, inplace=True)

Let's plot the schools again, this time applying a gradient color coding from red=low to green=high, instead of bins.

In [13]:
plot_df = plot_df.sort_values('AvgScore')

num_splits = plot_df.shape[0]*2
x = np.arange(num_splits)
ys = [i + x + (i*x)**2 for i in range(num_splits)]
colors_array = cm.gist_rainbow(np.linspace(0, 1, len(ys)))
hex_colors_array = [colors.rgb2hex(i) for i in colors_array]

plot_df['MarkerColor'] = hex_colors_array[0:plot_df.shape[0]]

map = folium.Map(location=coordinates('Green Bay, WI'), zoom_start=12, tiles='Stamen Toner')
for i,row in plot_df.iterrows():
    folium.CircleMarker((row['Latitude'],row['Longitude']),
                         radius=15,
                         color=row['MarkerColor'],
                         fill=True,
                         alpha=1,
                        popup=str(row['SCHOOL']+': '+str(round(row['AvgScore'],1)))).add_to(map)
map

Upon visual inspection, there appears to be a trend of decreasing scores for schools closer to the center of the city. In addition, poverty levels and crime rates tend to rise in neighborhoods closer to the center of the city, two factors which have an impact on students' school performance.

## Park Data
#### Names, Locations and Ratings
First, we'll use the Foursquare API to locate parks in the vicinity of each school. The following functions will be created to perform this task:
* Use Foursquare API to find venues near each school matching search term "Park"
* Use Foursquare API to confirm venue category is a park, and does not just have "Park" in the name
* Calculate distance between park and school
* Use Foursquare API to get rating of park


In [14]:
# Import libraries for this step
import requests
from pandas.io.json import json_normalize
from sklearn.metrics.pairwise import haversine_distances
from math import radians

In [15]:
# Define API credentials
CLIENT_ID = 'J1SNRIQCSENU5NGCTY0HSR1GJO4UWR1T54QKMLYXZRTSDLK0' # your Foursquare ID
CLIENT_SECRET = 'LJUGUOSP3N52R0ZBPLT4YR1H4L1VKNZLDP0AKMKSMQH2H5QV' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 20

We'll borrow this function from the previous lab to get venue categories.

In [16]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [17]:
pd.options.mode.chained_assignment = None  # default='warn'

This function will calculate distance between parks and schools.

In [18]:
# function that finds distance between two points on a sphere
# function adapted from scikit-learn documentation

def calculate_distance(loc1,loc2):
    loc1_in_radians = [radians(_) for _ in loc1]
    loc2_in_radians = [radians(_) for _ in loc2]
    result = haversine_distances([loc1_in_radians, loc2_in_radians])[1][0]
    
    return result * 6371000  # multiply by Earth radius to get meters

This function will look up park rating.

In [37]:
# function that gets rating from venue details

def get_rating(venue_id):
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    try:
        rating = result['response']['venue']['rating']
    except:
        rating = 'no rating'
    return rating

This function will look up parks and put them into a dataframe.

In [45]:
def get_parks(df,index):
    search_query = 'Park'
    radius = 1000
    limit=20
    lat,lng = list(df.loc[index][['Latitude','Longitude']])
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, search_query, radius, LIMIT)
    school = df.loc[index]['SCHOOL']

    # send the GET request
    results = requests.get(url).json()
    
    venues = results['response']['venues']
        
    nearby_parks = json_normalize(venues) # flatten JSON
    
    # filter columns
    filtered_columns = ['id','name', 'categories', 'location.lat', 'location.lng']
    #nearby_parks =nearby_parks.loc[:, filtered_columns]
    try:
        nearby_parks = nearby_parks[filtered_columns]
    except:
        print(f'0 parks found near {school}.')
        return None

    # filter the category for each row
    nearby_parks['categories'] = nearby_parks.apply(get_category_type, axis=1)
    
    # clean columns
    #nearby_parks.columns = [col.split(".")[-1] for col in nearby_parks.columns]
    
    # remove venues that are not parks
    nearby_parks = nearby_parks[nearby_parks['categories']=='Park']
    
    print(f'{nearby_parks.shape[0]} parks found near {school}.')

    nearby_parks['NearSchool'] = school
    nearby_parks['Distance'] = 0
    nearby_parks['Rating'] = 0
    for i,row in nearby_parks.iterrows():
        nearby_parks.at[[i],['Distance']] = calculate_distance([lat,lng], [nearby_parks.loc[i]['location.lat'],nearby_parks.loc[i]['location.lng']])
        nearby_parks.at[[i],['Rating']] = get_rating(nearby_parks.loc[i]['id'])
    

    return nearby_parks

Now that all the functions are defined, we can run through the list of schools and search for parks.

In [46]:
parks = pd.DataFrame(columns=['id','name','categories','location.lat','location.lng','NearSchool','Distance','Rating'])
for row in plot_df.index:
    df = get_parks(plot_df,row)
    parks = parks.append(df)

8 parks found near Howe El.
1 parks found near Eisenhower El.
5 parks found near Nicolet El.
5 parks found near Sullivan El.
3 parks found near Jefferson El.
1 parks found near Danz El.
6 parks found near Fort Howard El.
0 parks found near Lincoln El.
1 parks found near Elmore El.
2 parks found near Keller El.
0 parks found near Doty El.
2 parks found near Chappell El.
1 parks found near Martin El.
2 parks found near Baird El.
1 parks found near Beaumont El.
2 parks found near Tank El.
3 parks found near Jackson El.
0 parks found near King El.
1 parks found near Wilder El.
2 parks found near Webster El.
4 parks found near Kennedy El.
1 parks found near Langlade El.
0 parks found near Wequiock El.
2 parks found near MacArthur El.
1 parks found near McAuliffe El.


In [47]:
parks.reset_index(drop=True, inplace=True)

Now, let's see where those parks are on the map. We'll plot the schools again, then plot each of the parks with a purple dot.

In [48]:
map = folium.Map(location=coordinates('Green Bay, WI'), zoom_start=12, tiles='Stamen Toner')
for i,row in plot_df.iterrows():
    folium.CircleMarker((row['Latitude'],row['Longitude']),
                         radius=15,
                         color=row['MarkerColor'],
                         fill=True,
                         alpha=1,
                        popup=str(row['SCHOOL']+': '+str(round(row['AvgScore'],1)))).add_to(map)
for i,row in parks.iterrows():
    folium.CircleMarker((row['location.lat'],row['location.lng']),
                        radius=5,
                        color='purple',
                        fill=True,
                        alpha=1,
                        popup=str(row['name']+': '+str(row['Rating']))).add_to(map)
map

Interesting, there appear to be lots of parks near the low-scoring schools. Let's inspect the park data.

In [50]:
parks.describe(include='all')

,id,name,categories,location.lat,location.lng,NearSchool,Distance,Rating
count,54,54,54,54.000000,54.000000,54,54.000000,54
unique,39,38,1,NaN,NaN,21,NaN,6
top,5016dddee4b0c3d592c0acb0,John Muir Park,Park,NaN,NaN,Howe El,NaN,no rating
freq,3,4,54,NaN,NaN,8,NaN,48
mean,NaN,NaN,NaN,44.510249,-88.018229,NaN,713.718109,NaN
std,NaN,NaN,NaN,0.015470,0.037852,NaN,356.611566,NaN
min,NaN,NaN,NaN,44.461471,-88.087044,NaN,64.232900,NaN
25%,NaN,NaN,NaN,44.502508,-88.048328,NaN,444.986696,NaN
50%,NaN,NaN,NaN,44.510766,-88.017159,NaN,744.354502,NaN
75%,NaN,NaN,NaN,44.518741,-87.995321,NaN,1010.581398,NaN


I see a few things that will need to be cleaned up during the next week. 
* There are more IDs than Names, probably meaning some parks are duplicated in the Foursquare database. Duplicates will need to be removed
* Howe, the lowest scoring school, has the most parks. Will this still be true once duplicates are removed?
* 48 parks do not have a rating. This feature won't work for modeling.

Since most parks are not rated, I'll need to look elsewhere for data describing park characteristics. Perhaps the Green Bay Department of Parks, Recreation and Forestry website will have more information.

Thanks for reviewing my notebook. I look forward to your feedback.